# Gaussian elimination

In [1]:
import numpy as np
import scipy.linalg

In [47]:
def GEPP(A, b):
    '''
    Gaussian elimination with partial (row) pivoting
    input : A nonsingular and square matrix n x n 
            b vector n x 1
    output : x solution of the system A x = b
    '''
    # 1. Factorize A = PLU
    L, U, pT = LUPP(A)
    # 2. Solve P L U x = b
    Ptb = b[pT]
    # 3. Solve LUx = Pt b forward substitution
    y = forward_substitution(L, Ptb)
    # 4. Solve Ux = L^{-1} Pt b backward substitution
    x = backward_substitution(U,y)
    return x


def LUPP(Ainput):
    '''
    LU factorization with partial pivoting
    '''
    A = Ainput.copy()
    n = A.shape[0]
    pT = np.arange(0,n)
    for i in range(n-1):
        imax = abs(A[i:,i]).argmax() + i
        pT[[i,imax]] = pT[[imax,i]]
        if A[imax, i] == 0:
            raise ValueError("Matrix is singular.")
        elif imax != i:
            A[[i,imax],:] = A[[imax, i],:][:]
        A[(i+1):n,i][:] = (A[(i+1):n,i]/A[i,i])[:]
        A[(i+1):n, (i+1):n][:] = A[(i+1):n, (i+1):n]-np.outer(A[(i+1):n,i],A[i, (i+1):n])
    
    L = np.tril(A,-1)+np.eye(n)
    U = np.triu(A)
    return L, U, pT

def forward_substitution(L,b):
    '''
    Forward substitution algorithm for system L x = b
    input : L lower triangular matrix n x n
            b vector n x 1
    output: x solution of L x = b
    '''
    n = L.shape[0]
    x = np.zeros(n)
    x[0] = b[0]/L[0,0]
    for i in range(1,n):
        x[i] = (b[i] - L[i,0:i]@x[0:i])/L[i,i]
    return x

def backward_substitution(U,b):
    '''
    Backward substitution algorithm for system U x = b
    input : U upper tringular matrix n x n
            b vector n x 1
    output : x solution of U x = b
    '''
    n = U.shape[0]
    x = np.zeros(n)
    x[n-1] = b[n-1]/U[n-1,n-1]
    for i in range(n-2,-1,-1):
        x[i] = (b[i] - U[i,(i+1):n]@(x[(i+1):n]))/U[i,i]
    return x


# Example. 
Consider the system $Ax = b$ with 
\begin{equation}
A = 
\begin{pmatrix}
1 & 3 & 4 & 1 \\ 2 & 1 & 5 & 1 \\ 4 & 1 & 6 & 1 \\ 6 & 2 & 3 & 2
\end{pmatrix}
\qquad
b = 
\begin{pmatrix}
3\\2\\1\\3
\end{pmatrix}
\end{equation}

The solution of the system is
\begin{equation}
x= 
\begin{pmatrix}
-8/9\\0\\-1/9\\13/3
\end{pmatrix}
\end{equation}

In [51]:
A0 = np.array([[1,3,4,1],[2,1,5,1],[3,1,6,1],[6,2,3,2]],dtype=np.float64)
b0 = np.array([3,2,1,3], dtype=np.float64)
x1 = GEPP(A0,b0)
x2 = np.linalg.solve(A0,b0)
print(A0@x1, A0@x2)

[3. 2. 1. 3.] [3. 2. 1. 3.]
